# Python and MySQL Introduction
## Tutorial & Notes
* Created by: Tony Held (tony.held@gmail.com)  
* Created on: 2021-03-10 
* Source: https://realpython.com/python-mysql/

In [1]:
# **********************************************
#     Jupyter Interactive Mode Settings
# **********************************************
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"

# **********************************************
#     Allows autocomplete to work properly
# **********************************************
%config Completer.use_jedi = False

In [2]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
# usage pp.pprint(stuff)

In [3]:
def values(var, exclude_starting_with="_"):
    """Show variable attributes and values for diagnostic debugging purposes.
    
    Set exclude_starting_with to '' if you wish to include all attributes."""
    
    print(f'Variable type: {type(var)}')
    
    if exclude_starting_with:
        attrs = [i for i in dir(var) if not i.startswith(exclude_starting_with)]
    else:
        attrs = [i for i in dir(var)]
            
    for i in attrs:
        print(f'\n{i}:\n{"-"*20}\n{getattr(var, i)}')

In [4]:
# Function to show diagnostics for a cursor execution
def diag_execute(my_cursor, sql_statement):
    
    print(f'Executing: "{sql_statement}"\n')
    my_cursor.execute(sql_statement)
    
    print(f'Results of Execution:\n' +
           '--------------------')
    for row in my_cursor.fetchall():
        print(row)
    print()

## Get username and password for to access databases

In [ ]:
from getpass import getpass
my_username=input("Enter username: ")
my_password=getpass("Enter password: ")

## Establishing a Connection to MySQL Local Server
* Calling connect() gives you a connection (type: mysql.connector.connection_cext.CMySQLConnection')

In [6]:
import mysql.connector
from mysql.connector import connect, Error

try:
    with connect(
        host="localhost",
        user=my_username,
        password=my_password,
    ) as connection:
        print(type(connection))
        print(connection)
except Error as e:
    print(e)

<class 'mysql.connector.connection_cext.CMySQLConnection'>


In [7]:
with connect(host="localhost", 
             user=my_username, 
             password=my_password) as connection:
    print(type(connection))
    print(connection)

<class 'mysql.connector.connection_cext.CMySQLConnection'>


## Create a cursor on a connection object to be able to execute SQL statements
* A cursor is the same as if you were at a terminal cursor
* The results of a command you issue at a cursor is looped through as lines of text
* You can use this cursor to create a database

In [8]:
# Create cursor and leave it open - a little fast and loose, but less code :)

connection = connect(host="localhost", 
                     user=my_username, 
                     password=my_password)
cursor = connection.cursor()
print(type(cursor))
print(cursor)

<class 'mysql.connector.cursor_cext.CMySQLCursor'>
CMySQLCursor: (Nothing executed yet)


In [9]:
create_db_query = "CREATE DATABASE IF NOT EXISTS online_movie_rating"
cursor.execute(create_db_query)
print(cursor)

CMySQLCursor: CREATE DATABASE IF NOT EXISTS online_mov..


In [10]:
show_db_query = "SHOW DATABASES"
cursor.execute(show_db_query)
print(cursor)

CMySQLCursor: SHOW DATABASES


In [11]:
# loop through a cursor to see its output
for db in cursor:
    print(db)

('information_schema',)
('mysql',)
('online_movie_rating',)
('performance_schema',)
('sys',)


In [12]:
# Close cursor and connection
cursor.close()
connection.close()

## Open connection to newly formed database
* Connection approach is the same, but now you specify a database

In [13]:
connection = connect(host="localhost",
                     user=my_username,
                     password=my_password,
                     database="online_movie_rating");

In [14]:
# Create cursor without context manager to reduce code in examples
cursor = connection.cursor();

In [15]:
# Find out what database the cursor is connected to
cursor.execute('SELECT DATABASE()')
for row in cursor.fetchall():
    print(row)

('online_movie_rating',)


In [16]:
# Display tables in the connected database
cursor.execute('SHOW TABLES')
for row in cursor.fetchall():
    print(row)

('movies',)
('ratings',)
('reviewers',)


In [17]:
# Delete all the table if you are running this notebook a second time
diag_execute(cursor, 'DROP TABLE IF EXISTS movies, ratings, reviewers')

Executing: "DROP TABLE IF EXISTS movies, ratings, reviewers"

Results of Execution:
--------------------



In [18]:
# Queries to create data tables

create_movies_table_query = """
CREATE TABLE IF NOT EXISTS movies(
    id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(100),
    release_year YEAR(4),
    genre VARCHAR(100),
    collection_in_mil INT
)
""";

create_reviewers_table_query = """
CREATE TABLE IF NOT EXISTS reviewers (
    id INT AUTO_INCREMENT PRIMARY KEY,
    first_name VARCHAR(100),
    last_name VARCHAR(100)
)
""";

create_ratings_table_query = """
CREATE TABLE IF NOT EXISTS ratings (
    movie_id INT,
    reviewer_id INT,
    rating DECIMAL(2,1),
    FOREIGN KEY(movie_id) REFERENCES movies(id),
    FOREIGN KEY(reviewer_id) REFERENCES reviewers(id),
    PRIMARY KEY(movie_id, reviewer_id)
)
""";

### Execute sql statements at the cursor

* Make sure to commit() after your desired executions
* You can loop through the cursor to see the last execution result
```
 for row in cursor:
     print(line)
```
* If you try to use the cursor while there are unread lines in its output, you get an error
* It may be helpful to store cursor output to local variable with fetch all
and then loop through the local variable
```
    result = cursor.fetchall()
    for row in result:
        print(row)
```

In [19]:
# Add new table structures
cursor.execute(create_movies_table_query)
cursor.execute(create_reviewers_table_query)
cursor.execute(create_ratings_table_query)
connection.commit()

In [20]:
# Same as above, with some formating
diag_execute(cursor, 'DESCRIBE movies')

Executing: "DESCRIBE movies"

Results of Execution:
--------------------
('id', b'int', 'NO', 'PRI', None, 'auto_increment')
('title', b'varchar(100)', 'YES', '', None, '')
('release_year', b'year', 'YES', '', None, '')
('genre', b'varchar(100)', 'YES', '', None, '')
('collection_in_mil', b'int', 'YES', '', None, '')



In [21]:
# Alter a table field with a query
alter_table_query = """
ALTER TABLE movies
MODIFY COLUMN collection_in_mil DECIMAL(4,1)
""";

In [22]:
print(f'Before the table alteration')
diag_execute(cursor, 'DESCRIBE movies')
diag_execute(cursor, alter_table_query)

print(f'After the table alteration')
diag_execute(cursor, 'DESCRIBE movies')

Before the table alteration
Executing: "DESCRIBE movies"

Results of Execution:
--------------------
('id', b'int', 'NO', 'PRI', None, 'auto_increment')
('title', b'varchar(100)', 'YES', '', None, '')
('release_year', b'year', 'YES', '', None, '')
('genre', b'varchar(100)', 'YES', '', None, '')
('collection_in_mil', b'int', 'YES', '', None, '')

Executing: "
ALTER TABLE movies
MODIFY COLUMN collection_in_mil DECIMAL(4,1)
"

Results of Execution:
--------------------

After the table alteration
Executing: "DESCRIBE movies"

Results of Execution:
--------------------
('id', b'int', 'NO', 'PRI', None, 'auto_increment')
('title', b'varchar(100)', 'YES', '', None, '')
('release_year', b'year', 'YES', '', None, '')
('genre', b'varchar(100)', 'YES', '', None, '')
('collection_in_mil', b'decimal(4,1)', 'YES', '', None, '')



## Populate the database with sample data
* Use execute if you are including column names and data together
* Use executemany if you are including a framework for subsequent executions

In [23]:
insert_movies_query = """
INSERT INTO movies (title, release_year, genre, collection_in_mil)
VALUES
    ("Forrest Gump", 1994, "Drama", 330.2),
    ("3 Idiots", 2009, "Drama", 2.4),
    ("Eternal Sunshine of the Spotless Mind", 2004, "Drama", 34.5),
    ("Good Will Hunting", 1997, "Drama", 138.1),
    ("Skyfall", 2012, "Action", 304.6),
    ("Gladiator", 2000, "Action", 188.7),
    ("Black", 2005, "Drama", 3.0),
    ("Titanic", 1997, "Romance", 659.2),
    ("The Shawshank Redemption", 1994, "Drama",28.4),
    ("Udaan", 2010, "Drama", 1.5),
    ("Home Alone", 1990, "Comedy", 286.9),
    ("Casablanca", 1942, "Romance", 1.0),
    ("Avengers: Endgame", 2019, "Action", 858.8),
    ("Night of the Living Dead", 1968, "Horror", 2.5),
    ("The Godfather", 1972, "Crime", 135.6),
    ("Haider", 2014, "Action", 4.2),
    ("Inception", 2010, "Adventure", 293.7),
    ("Evil", 2003, "Horror", 1.3),
    ("Toy Story 4", 2019, "Animation", 434.9),
    ("Air Force One", 1997, "Drama", 138.1),
    ("The Dark Knight", 2008, "Action",535.4),
    ("Bhaag Milkha Bhaag", 2013, "Sport", 4.1),
    ("The Lion King", 1994, "Animation", 423.6),
    ("Pulp Fiction", 1994, "Crime", 108.8),
    ("Kai Po Che", 2013, "Sport", 6.0),
    ("Beasts of No Nation", 2015, "War", 1.4),
    ("Andadhun", 2018, "Thriller", 2.9),
    ("The Silence of the Lambs", 1991, "Crime", 68.2),
    ("Deadpool", 2016, "Action", 363.6),
    ("Drishyam", 2015, "Mystery", 3.0)
""";

In [24]:
insert_reviewers_query = """
INSERT INTO reviewers
(first_name, last_name)
VALUES ( %s, %s )
"""
reviewers_records = [
    ("Chaitanya", "Baweja"),
    ("Mary", "Cooper"),
    ("John", "Wayne"),
    ("Thomas", "Stoneman"),
    ("Penny", "Hofstadter"),
    ("Mitchell", "Marsh"),
    ("Wyatt", "Skaggs"),
    ("Andre", "Veiga"),
    ("Sheldon", "Cooper"),
    ("Kimbra", "Masters"),
    ("Kat", "Dennings"),
    ("Bruce", "Wayne"),
    ("Domingo", "Cortes"),
    ("Rajesh", "Koothrappali"),
    ("Ben", "Glocker"),
    ("Mahinder", "Dhoni"),
    ("Akbar", "Khan"),
    ("Howard", "Wolowitz"),
    ("Pinkie", "Petit"),
    ("Gurkaran", "Singh"),
    ("Amy", "Farah Fowler"),
    ("Marlon", "Crafford"),
];

In [25]:
insert_ratings_query = """
INSERT INTO ratings
(rating, movie_id, reviewer_id)
VALUES ( %s, %s, %s)
"""
ratings_records = [
    (6.4, 17, 5), (5.6, 19, 1), (6.3, 22, 14), (5.1, 21, 17),
    (5.0, 5, 5), (6.5, 21, 5), (8.5, 30, 13), (9.7, 6, 4),
    (8.5, 24, 12), (9.9, 14, 9), (8.7, 26, 14), (9.9, 6, 10),
    (5.1, 30, 6), (5.4, 18, 16), (6.2, 6, 20), (7.3, 21, 19),
    (8.1, 17, 18), (5.0, 7, 2), (9.8, 23, 3), (8.0, 22, 9),
    (8.5, 11, 13), (5.0, 5, 11), (5.7, 8, 2), (7.6, 25, 19),
    (5.2, 18, 15), (9.7, 13, 3), (5.8, 18, 8), (5.8, 30, 15),
    (8.4, 21, 18), (6.2, 23, 16), (7.0, 10, 18), (9.5, 30, 20),
    (8.9, 3, 19), (6.4, 12, 2), (7.8, 12, 22), (9.9, 15, 13),
    (7.5, 20, 17), (9.0, 25, 6), (8.5, 23, 2), (5.3, 30, 17),
    (6.4, 5, 10), (8.1, 5, 21), (5.7, 22, 1), (6.3, 28, 4),
    (9.8, 13, 1)];

In [26]:
cursor.execute(insert_movies_query)
cursor.executemany(insert_reviewers_query, reviewers_records)
cursor.executemany(insert_ratings_query, ratings_records)
connection.commit()

## Inspect the database

In [27]:
select_movies_query = "SELECT * FROM movies LIMIT 0,5"
diag_execute(cursor, select_movies_query)

Executing: "SELECT * FROM movies LIMIT 0,5"

Results of Execution:
--------------------
(1, 'Forrest Gump', 1994, 'Drama', Decimal('330.2'))
(2, '3 Idiots', 2009, 'Drama', Decimal('2.4'))
(3, 'Eternal Sunshine of the Spotless Mind', 2004, 'Drama', Decimal('34.5'))
(4, 'Good Will Hunting', 1997, 'Drama', Decimal('138.1'))
(5, 'Skyfall', 2012, 'Action', Decimal('304.6'))



In [28]:
select_movies_query = "SELECT * FROM movies LIMIT 5,5"
diag_execute(cursor, select_movies_query)

Executing: "SELECT * FROM movies LIMIT 5,5"

Results of Execution:
--------------------
(6, 'Gladiator', 2000, 'Action', Decimal('188.7'))
(7, 'Black', 2005, 'Drama', Decimal('3.0'))
(8, 'Titanic', 1997, 'Romance', Decimal('659.2'))
(9, 'The Shawshank Redemption', 1994, 'Drama', Decimal('28.4'))
(10, 'Udaan', 2010, 'Drama', Decimal('1.5'))



In [29]:
select_movies_query = "SELECT * FROM movies LIMIT 5"
diag_execute(cursor, select_movies_query)

Executing: "SELECT * FROM movies LIMIT 5"

Results of Execution:
--------------------
(1, 'Forrest Gump', 1994, 'Drama', Decimal('330.2'))
(2, '3 Idiots', 2009, 'Drama', Decimal('2.4'))
(3, 'Eternal Sunshine of the Spotless Mind', 2004, 'Drama', Decimal('34.5'))
(4, 'Good Will Hunting', 1997, 'Drama', Decimal('138.1'))
(5, 'Skyfall', 2012, 'Action', Decimal('304.6'))



In [30]:
select_movies_query = """
SELECT title, collection_in_mil
FROM movies
WHERE collection_in_mil > 300
ORDER BY collection_in_mil DESC
"""
diag_execute(cursor, select_movies_query)

Executing: "
SELECT title, collection_in_mil
FROM movies
WHERE collection_in_mil > 300
ORDER BY collection_in_mil DESC
"

Results of Execution:
--------------------
('Avengers: Endgame', Decimal('858.8'))
('Titanic', Decimal('659.2'))
('The Dark Knight', Decimal('535.4'))
('Toy Story 4', Decimal('434.9'))
('The Lion King', Decimal('423.6'))
('Deadpool', Decimal('363.6'))
('Forrest Gump', Decimal('330.2'))
('Skyfall', Decimal('304.6'))



In [31]:
select_movies_query = """
SELECT CONCAT(title, " (", release_year, ")"),
      collection_in_mil
FROM movies
ORDER BY collection_in_mil DESC
LIMIT 5
"""
diag_execute(cursor, select_movies_query)

Executing: "
SELECT CONCAT(title, " (", release_year, ")"),
      collection_in_mil
FROM movies
ORDER BY collection_in_mil DESC
LIMIT 5
"

Results of Execution:
--------------------
('Avengers: Endgame (2019)', Decimal('858.8'))
('Titanic (1997)', Decimal('659.2'))
('The Dark Knight (2008)', Decimal('535.4'))
('Toy Story 4 (2019)', Decimal('434.9'))
('The Lion King (1994)', Decimal('423.6'))



In [32]:
select_movies_query = """
SELECT CONCAT(title, " (", release_year, ")"),
      collection_in_mil
FROM movies
ORDER BY collection_in_mil DESC
"""
diag_execute(cursor, select_movies_query)

Executing: "
SELECT CONCAT(title, " (", release_year, ")"),
      collection_in_mil
FROM movies
ORDER BY collection_in_mil DESC
"

Results of Execution:
--------------------
('Avengers: Endgame (2019)', Decimal('858.8'))
('Titanic (1997)', Decimal('659.2'))
('The Dark Knight (2008)', Decimal('535.4'))
('Toy Story 4 (2019)', Decimal('434.9'))
('The Lion King (1994)', Decimal('423.6'))
('Deadpool (2016)', Decimal('363.6'))
('Forrest Gump (1994)', Decimal('330.2'))
('Skyfall (2012)', Decimal('304.6'))
('Inception (2010)', Decimal('293.7'))
('Home Alone (1990)', Decimal('286.9'))
('Gladiator (2000)', Decimal('188.7'))
('Good Will Hunting (1997)', Decimal('138.1'))
('Air Force One (1997)', Decimal('138.1'))
('The Godfather (1972)', Decimal('135.6'))
('Pulp Fiction (1994)', Decimal('108.8'))
('The Silence of the Lambs (1991)', Decimal('68.2'))
('Eternal Sunshine of the Spotless Mind (2004)', Decimal('34.5'))
('The Shawshank Redemption (1994)', Decimal('28.4'))
('Kai Po Che (2013)', Decimal('

In [33]:
select_movies_query = """
SELECT title, 
AVG(rating) as average_rating,
COUNT(rating) as number_ratings
FROM ratings
LEFT JOIN movies
    ON movies.id = ratings.movie_id
GROUP BY movie_id
ORDER BY average_rating DESC, title
"""
diag_execute(cursor, select_movies_query)

Executing: "
SELECT title, 
AVG(rating) as average_rating,
COUNT(rating) as number_ratings
FROM ratings
LEFT JOIN movies
    ON movies.id = ratings.movie_id
GROUP BY movie_id
ORDER BY average_rating DESC, title
"

Results of Execution:
--------------------
('Night of the Living Dead', Decimal('9.90000'), 1)
('The Godfather', Decimal('9.90000'), 1)
('Avengers: Endgame', Decimal('9.75000'), 2)
('Eternal Sunshine of the Spotless Mind', Decimal('8.90000'), 1)
('Beasts of No Nation', Decimal('8.70000'), 1)
('Gladiator', Decimal('8.60000'), 3)
('Home Alone', Decimal('8.50000'), 1)
('Pulp Fiction', Decimal('8.50000'), 1)
('Kai Po Che', Decimal('8.30000'), 2)
('The Lion King', Decimal('8.16667'), 3)
('Air Force One', Decimal('7.50000'), 1)
('Inception', Decimal('7.25000'), 2)
('Casablanca', Decimal('7.10000'), 2)
('Udaan', Decimal('7.00000'), 1)
('Drishyam', Decimal('6.84000'), 5)
('The Dark Knight', Decimal('6.82500'), 4)
('Bhaag Milkha Bhaag', Decimal('6.66667'), 3)
('The Silence of the Lamb

In [34]:
select_movies_query = """
SELECT CONCAT(first_name, " ", last_name), COUNT(rating) as num
FROM reviewers
INNER JOIN ratings
    ON reviewers.id = ratings.reviewer_id
GROUP BY reviewer_id
ORDER BY num DESC
"""
diag_execute(cursor, select_movies_query)

Executing: "
SELECT CONCAT(first_name, " ", last_name), COUNT(rating) as num
FROM reviewers
INNER JOIN ratings
    ON reviewers.id = ratings.reviewer_id
GROUP BY reviewer_id
ORDER BY num DESC
"

Results of Execution:
--------------------
('Mary Cooper', 4)
('Chaitanya Baweja', 3)
('Penny Hofstadter', 3)
('Domingo Cortes', 3)
('Akbar Khan', 3)
('Howard Wolowitz', 3)
('Pinkie Petit', 3)
('John Wayne', 2)
('Thomas Stoneman', 2)
('Mitchell Marsh', 2)
('Sheldon Cooper', 2)
('Kimbra Masters', 2)
('Rajesh Koothrappali', 2)
('Ben Glocker', 2)
('Mahinder Dhoni', 2)
('Gurkaran Singh', 2)
('Andre Veiga', 1)
('Kat Dennings', 1)
('Bruce Wayne', 1)
('Amy Farah Fowler', 1)
('Marlon Crafford', 1)



In [35]:
update_query = """
UPDATE reviewers
SET   last_name = "Cooper"
WHERE first_name = "Amy"
"""
diag_execute(cursor, update_query)

Executing: "
UPDATE reviewers
SET   last_name = "Cooper"
WHERE first_name = "Amy"
"

Results of Execution:
--------------------

